TODO

generic feature showcase showing genesis features
demonstrate robot viewer
video recording
point to next notebook for intro
point to other notebook for advanced uses

In [47]:
from pathlib import Path
import random 

from manipylator import HeadlessVisualRobot, VisualRobot
from manipylator.utils import render_robot_from_template

# ManiPyLator Basics
ManiPyLator wraps around two main robotics libraries, robotics-toolbox-python & Genesis and tries to provide an interface to both simulated and analytical robotics. Below is an example of loading an arbitrary URDF and a forward kinematics sanity check with both libraries:

In [2]:
with render_robot_from_template("robots/empiric") as robot_urdf:
    manny = HeadlessVisualRobot(robot_urdf)
print(manny.model)

[I 07/23/25 01:18:44.679 468] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Genesis] [01:18:51] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [01:18:51] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [01:18:51] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [01:18:51] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.
[Genesis] [01:18:52] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [01:18:52] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [01:18:52] [WARNING] Scene.show_FPS is deprecated. Please use Scene.profiling_options.show_FPS
[Genesis] [01:18:52] [INFO] Scene <f118e71> created.
[Genesis] [01:18:52] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <7d700f1>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [01:18:52] [INFO] Adding <gs.RigidEntity>. idx:

[Genesis] [01:18:54] [INFO] Falling back to legacy URDF parser. Default values of physics properties may be off.
[Genesis] [01:18:54] [INFO] Applying offset to base link's pose with user provided value in morph.
[Genesis] [01:19:00] [INFO] Building scene <f118e71>...
[Genesis] [01:19:19] [INFO] Compiling simulation kernels...
[Genesis] [01:19:34] [INFO] Building visualizer...
[Genesis] [01:19:35] [INFO] Software rendering context detected. Shadows and plane reflection not supported. Beware rendering will be extremely slow.
ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    

For an arbitrary poes, we'll calculate the forward kinematics using both RTB and Genesis and compare:

In [68]:
arbitrary_pose = [random.uniform(-1, 1) for _ in range(6)]
print(arbitrary_pose)

[-0.23398091331366788, -0.183635696332614, 0.27363530336807496, 0.9624579321551772, 0.2651592298750267, -0.6880417967579759]


In [69]:
forward_kinematics_rtb = manny.model.fkine(arbitrary_pose)
forward_kinematics_rtb

  -0.02559   0.1242    0.9919   -0.1591    
   0.9805    0.1966    0.0006791  0.1192    
  -0.1949    0.9726   -0.1268    0.7708    
   0         0         0         1         


In [70]:
translation_rtb = forward_kinematics_rtb.t
translation_rtb

array([-0.1590787 ,  0.119162  ,  0.77075704])

In [71]:
simulated_robot = manny.visualizer.robot
ee = simulated_robot.get_link('end_effector')
scene = manny.visualizer.scene

simulated_robot.set_dofs_position(arbitrary_pose)
scene.step()

translation_gs = ee.get_pos().cpu()
translation_gs

tensor([-0.1591,  0.1192,  0.7708])

In [72]:
from numpy import allclose
allclose(translation_rtb, translation_gs, atol=1e-5)

True

Both approaches should return True above, which means they calculate comparable solutios down to the 10 micron range

# Where To Now?
The above demonstrates the two available interfaces in ManiPylator. Based on your interests you might continue to explore our visual robotics demonstration or maybe try to derive a symbolic result. For whatever path you're interested in, check out our Robotics resources page.

For beginners, we recommend to:


For experienced robotocists, check out our